In [ ]:
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt
%matplotlib inline

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.getcwd())
os.chdir("/content/drive/My Drive/CS505-Project/gpt2-outputs")

/content/drive/MyDrive/CS505-Project/data/combined_data


In [ ]:
files = os.listdir(os.getcwd())
files

['gen_abe.txt',
 'gen_obama.txt',
 'gen_trump.txt',
 'gen_wc.txt',
 'gen_hitler.txt',
 'gen_biden.txt',
 'gen_kennedy.txt',
 'gen_washington.txt',
 'gen_hillary.txt']

In [ ]:
from collections import defaultdict
outputs = defaultdict()
for i in files:
  with open(i, "r") as f:
    outputs[i.split("_")[1].split(".")[0]]=f.read()

In [ ]:
#Getting ground truth

import json

gt=defaultdict(list)
print(os.getcwd())
os.chdir("../data/combined_data")

for json_file in os.listdir():
      with open(json_file, "r") as f:
        temp = []
        all = json.load(f)
        if type(all) is list:
          for i in all:
            temp.append(i['Text'])
          gt[json_file.split(".")[0]] = temp[:5]
        elif 'Items' in all:
          for i in all['Items']:
            temp.append(i['transcript'])
          gt[json_file.split(".")[0]] = temp[:5]
        elif 'speeches' in all:
          for i in all['speeches']:
            temp.append(i['transcript'])
          gt[json_file.split(".")[0]] = temp[:5]

/content/drive/My Drive/CS505-Project/gpt2-outputs


In [ ]:
gt.keys()

dict_keys(['washington', 'kennedy', 'biden', 'hitler', 'abe', 'wc', 'hillary', 'obama', 'trump'])

In [ ]:
len(gt['abe'])

5

In [ ]:
for key in gt:
  gt[key] = " ".join(gt[key])

In [ ]:
gt.keys()

dict_keys(['washington', 'kennedy', 'biden', 'hitler', 'abe', 'wc', 'hillary', 'obama', 'trump'])

#Evaluation Metrics

1. Cosine Similarity

In [ ]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [ ]:
for key in gt:
  text1 = gt[key]
  text2 = outputs[key]

  vector1 = text_to_vector(text1)
  vector2 = text_to_vector(text2)

  cosine = get_cosine(vector1, vector2)

  print(f"Cosine Similarity of generated {key} text ={cosine}")

Cosine Similarity of generated washington text =0.9865454097570104
Cosine Similarity of generated kennedy text =0.9732442770880877
Cosine Similarity of generated biden text =0.9528666579841972
Cosine Similarity of generated hitler text =0.9761071484951617
Cosine Similarity of generated abe text =0.9767988023758633
Cosine Similarity of generated wc text =0.9810289209027907
Cosine Similarity of generated hillary text =0.9619610966234926
Cosine Similarity of generated obama text =0.9639489525120548
Cosine Similarity of generated trump text =0.9578613758426511


2. BLEU Score

In [ ]:
import nltk

for key in gt:

  list_of_references=[]
  list_of_hypotheses=[]

  list_of_references.append(gt[key].split()) # list of references for all sentences in corpus.
  list_of_hypotheses.append(outputs[key].split()) # list of hypotheses that corresponds to list of references.

  bleu = nltk.translate.bleu_score.corpus_bleu(list_of_references, list_of_hypotheses)
  print(f"BLEU Score of generated {key} text ={bleu}")

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

BLEU Score of generated washington text =3.626178001439019e-232
BLEU Score of generated kennedy text =5.176857472917499e-156
BLEU Score of generated biden text =5.7644671367713944e-232
BLEU Score of generated hitler text =4.857652242153452e-232
BLEU Score of generated abe text =4.671265459790935e-232
BLEU Score of generated wc text =3.792346690660694e-232
BLEU Score of generated hillary text =5.3146296728277966e-232
BLEU Score of generated obama text =4.8451938810320765e-232
BLEU Score of generated trump text =4.8544115075475724e-232


3. ROUGE Score

In [ ]:
!pip install evaluate
!pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8cc682d88c4c4fb83572cf66b782f0ad147589d413ca3d0464671a77388be60c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import evaluate
rouge = evaluate.load('rouge')

for key in gt:

  list_of_references=[gt[key]] # list of references for all sentences in corpus.
  list_of_hypotheses=[outputs[key]] # list of hypotheses that corresponds to list of references.
  # print(list_of_hypotheses)
  results = rouge.compute(predictions=list_of_hypotheses, references=list_of_references)
  print(f"ROUGE Scores of generated {key} text ={results}")

ROUGE Scores of generated washington text ={'rouge1': 0.7745311818578282, 'rouge2': 0.3173391494002181, 'rougeL': 0.20606192760575665, 'rougeLsum': 0.6332315743567379}
ROUGE Scores of generated kennedy text ={'rouge1': 0.24552928607924196, 'rouge2': 0.1460491475821062, 'rougeL': 0.0946630126805704, 'rougeLsum': 0.22778577732361005}
ROUGE Scores of generated biden text ={'rouge1': 0.12161490121614903, 'rouge2': 0.09305059456526653, 'rougeL': 0.0581400605814006, 'rougeLsum': 0.11944482119444823}
ROUGE Scores of generated hitler text ={'rouge1': 0.14403583883864016, 'rouge2': 0.10315592480222305, 'rougeL': 0.06481612747738807, 'rougeLsum': 0.06481612747738807}
ROUGE Scores of generated abe text ={'rouge1': 0.16015192277259058, 'rouge2': 0.11413920805241667, 'rougeL': 0.0698844753916759, 'rougeLsum': 0.0698844753916759}
ROUGE Scores of generated wc text ={'rouge1': 0.2861094208109753, 'rouge2': 0.16863070539419087, 'rougeL': 0.10731869226088399, 'rougeLsum': 0.10731869226088399}
ROUGE Scor